# Home task 

1) Replicate Simple recommender implementation 
1) (optional) Replicate the the content based  recommender implementation 

[Beginner Tutorial: Recommender Systems in Python](https://www.datacamp.com/community/tutorials/recommender-systems-python?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034358&utm_targetid=aud-748597547652:dsa-473406569915&utm_loc_interest_ms=&utm_loc_physical_ms=1012865&gclid=Cj0KCQjwsZKJBhC0ARIsAJ96n3XK-0y5uKGhO4w7V-A3nvj7WZlIg9NVQ8aeCLYKiEqhcb44rtw9qDoaAmeLEALw_wcB)

In [ ]:
import pandas as pd

### Simple recommender

In [ ]:
# Load Movies Metadata
metadata = pd.read_csv('./dataset/movies_metadata.csv', low_memory=False)

# Print the first three rows
metadata.head(3)

#### WeightedRating(WR)=(vv+m⋅R)+(mv+m⋅C)
- v is the number of votes for the movie;

- m is the minimum votes required to be listed in the chart;

- R is the average rating of the movie;

- C is the mean vote across the whole report.

In [ ]:
# Calculate mean of vote average column
C = metadata['vote_average'].mean()
print(C)

In [ ]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.90)
print(m)

In [ ]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

In [ ]:
metadata.shape

In [ ]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [ ]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

#### Model evaluation

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
# calculated accuracy
accuracy = (q_movies["score"].mean()/q_movies["vote_average"].mean()) * 100
print(f"Model accuracy: {round(accuracy, 2)}%")

# r2_score
r2_score_ = r2_score(q_movies['vote_average'], q_movies['score']) * 100
print(f"Model r2_score - {round(r2_score_, 2)}%")

### Content-based recommender

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
# Load Movies Metadata
metadata = pd.read_csv('./dataset/movies_metadata.csv', low_memory=False)

# Print plot overviews of the first 5 movies.
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [34]:
# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

# Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

# Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75827)

In [35]:
# Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[5000:5010]

['avails',
 'avaks',
 'avalanche',
 'avalanches',
 'avallone',
 'avalon',
 'avant',
 'avanthika',
 'avanti',
 'avaracious']

In [36]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# output shape
cosine_sim.shape

MemoryError: Unable to allocate 15.4 GiB for an array with shape (45466, 45466) and data type float64

In [ ]:
cosine_sim[1]